In [ ]:
%cd ..

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

from data.datasources import VOC2007
from utils.image import visualize
from data.transforms import Denormalize
from data.datasets import ImageDataset_Yolo

In [ ]:
datasource = VOC2007(
    "/home/hien/Documents/datasets",
    download=False,
    extract=False,
    use_tqdm=True,
    bbox_type="albumentations",
)

In [ ]:
dataset = ImageDataset_Yolo(
    data=datasource.get_data(phase="train"),
    transform=A.Compose(
        [
            A.Resize(416, 416),
            A.HorizontalFlip(p=0.5),
            A.Rotate(limit=45, p=0.5),
            A.PadIfNeeded(
                min_height=416 + 10,
                min_width=416 + 10,
            ),
            A.RandomCrop(416, 416),
            A.RandomSizedCrop(
                min_max_height=(
                    int(416 * 2.0 / 3.0),
                    416,
                ),
                height=416,
                width=416,
                p=1,
            ),
            A.Equalize(mode="cv", p=0.8),
            A.HueSaturationValue(
                hue_shift_limit=20, sat_shift_limit=20, val_shift_limit=20, p=0.5
            ),
            A.ToGray(p=0.5),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ],
        bbox_params=A.BboxParams(
            format="albumentations",
            min_area=4500,
            min_visibility=0.1,
            label_fields=["category_ids"],
        ),
    ),
    bbox_type="albumentations",
    mosaic_prob=0.5,
)

In [ ]:

for i in range(len(dataset)):
    img, boxes, labels, difficulties = dataset.mosaic_test(i)

    transformed_image = (
        Denormalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])(img)
        .permute(1, 2, 0)
        .numpy()
    )

    visualize(
        transformed_image,
        boxes,
        labels,
        datasource.revert_label_map,
        bbox_type="albumentations",
    )

    if i == 10:
        break